In [2]:
import os
import pandas as pd
import numpy as np
import datetime
import plotly.express as px
import plotly
import plotly.graph_objects as go
import argparse
import pyarrow
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from plotly.subplots import make_subplots
import plotly.figure_factory as ff
from functools import reduce
from datetime import datetime, timedelta, date

In [3]:
folder_path = "path/to/data"
outputs_path = "output/path"

In [7]:
class weekly_report:
    """

    This class loads sensor data at hourly level and at daily level and creates
    biweekly dashboard to monitor the violations and speed of each vehicle.

    """

    def __init__(self,
                 folder_path,
                 outputs_path,
                 year,
                 month,
                 date
                 ):
        """
       Parameters
       ----------
       folder_path : Folder containing sensor data at hourly level and daily level 
       outputs_path : Folder to save outputs. 
       year: enter the year 
       month: enter the month
       date: enter the date

       """

        self.folder_path = folder_path
        self.sensor_day_path = os.path.join(
            self.folder_path, "sensor_day.gz.parquet")
        self.sensor_dayhr_path = os.path.join(
            self.folder_path, "sensor_dayhr.gz.parquet")

        self.outputs_path = outputs_path
        if not os.path.exists(self.outputs_path):
            os.mkdir(self.outputs_path)

        self.year = year
        self.month = month
        self.date = date

        print('Loading data...')
        self.load_data()

        print('Aggregating data...')
        self.agg_all_data()
        self.agg_past_twowk_data()

    # -------------------------------------------------------------------------
    # Load sensor data

    def load_data(self):
        self.sensor_dayhr_raw = pd.read_parquet(self.sensor_dayhr_path)
        self.sensor_day_raw = pd.read_parquet(self.sensor_day_path)

        # preprocess function: add week variable to each entry
        def preprocess(df):
            df = df.sort_values(['date'])
            first_date = df.date.unique()[0]
            last_date = df.date.unique()[-1]
            num_days = abs(last_date - first_date).days + 1
            num_weeks = (num_days // 7) + 1 if (num_days %
                                                7) != 0 else num_days // 7
            week_bin = pd.date_range(
                df['date'].unique()[0], periods=num_weeks + 1, freq='168H')
            labels = [i for i in range(1, len(week_bin))]
            df["week"] = pd.cut(df['date'].astype('datetime64[ns]'),
                                week_bin, right=False, include_lowest=True, labels=labels)

            df['route'] = df['route'].fillna('Missing')

            return df

        self.sensor_dayhr = preprocess(self.sensor_dayhr_raw)
        self.sensor_day = preprocess(self.sensor_day_raw)


    # -------------------------------------------------------------------------
    # calculate the number of total violations(medium violation + extreme violation) 
    # per week and the number of extreme violations per week for each vehicle over 
    # the whole period of time 
    def agg_all_data(self):
        self.totext_vio_perwk = self.sensor_day.groupby(['regno_clean', 'week']).\
            apply(lambda x:pd.Series({'total_vio': x['N_valueg_above0_3'].sum(),
                    'ext_vio': x['N_valueg_above0_4'].sum()})).reset_index()
    
    
    
    # -------------------------------------------------------------------------
    # only use past two week data to calculate: 
    # (1) the number of violation for each violation type for each vehicle
    # (2) the number of total violations(medium violation + extreme violation) 
    # per day and the number of extreme violations per day for each vehicle 
    # (3) the number of violations per 100km for all vehicles
    # (4) the number of violations per 100km for all vehicles(without outlier in distance)
    # (5) 85th pert of speed each day each hr for each vehicle within same route
    # (6) a summery table including the number of violation type; first & last online date in sensor data
    # and first & last online date in eco data; and numbe of online days in past two weeks
    def agg_past_twowk_data(self):
        
        self.start_date = date(self.year,self.month,self.date) - timedelta(days = 13)
        self.past2wk_sensor_day = self.sensor_day[(self.sensor_day.date <= date(self.year,self.month,self.date)) &\
                                             (self.sensor_day.date >= self.start_date)]
        
        self.past2wk_sensor_dayhr = self.sensor_dayhr[(self.sensor_dayhr.date <= date(self.year,self.month,self.date)) &\
                                             (self.sensor_dayhr.date >= self.start_date)]
        
        
        self.past2wk_viotype_cnt = self.past2wk_sensor_day.groupby(['regno_clean']).apply(lambda x: \
            pd.Series({'brake': x['N_valueg_above0_3_brake'].sum(),
                      'turn': x['N_valueg_above0_3_turn'].sum(),
                      'acceleration': x['N_valueg_above0_3_acceleration'].sum()})).reset_index()
        
        self.past2wk_totext_vio_perday = self.past2wk_sensor_day.groupby(['regno_clean','route' ,'date']).apply(lambda x: \
                            pd.Series({'total_vio': x['N_valueg_above0_3'].sum(),
                            'ext_vio': x['N_valueg_above0_4'].sum()})).reset_index()
            
    
        
        def complete_time(df, timevar = 'datetime_eat', timefreq = 'H'):
           
            df[timevar] = df[timevar].astype('datetime64[ns, Africa/Nairobi]')
                
            df = df.sort_values(timevar)
            full_time_range = pd.date_range(df[timevar].unique()[0],  
                                    df[timevar].unique()[-1], 
                                            freq = timefreq)
    
            cpl_time = pd.DataFrame(full_time_range, columns = [timevar])
            cpl_df = df.merge(cpl_time, how = 'outer')
     
            cpl_df['regno_clean'] = cpl_df['regno_clean'].fillna(cpl_df['regno_clean'][0])
            cpl_df['route'] = cpl_df['route'].fillna(cpl_df['route'][0])
            
            if timevar == 'date':
                cpl_df['date'] = pd.to_datetime(cpl_df['date']).dt.date
    
            return cpl_df
    
    
        def complete_time_perveh(df, timevar = 'datetime_eat', timefreq = 'H'):
           temp = pd.DataFrame()
           uniq_vh = df.regno_clean.unique()
           for vh in uniq_vh:
               sample_vh = df[df.regno_clean == vh]
               sample_vh = sample_vh.sort_values([timevar])
               cpl_time_vh = complete_time(sample_vh, timevar, timefreq)
               temp =temp.append(cpl_time_vh)
   
           temp = temp.sort_values(['regno_clean', timevar])
           
           return temp
       
      
        self.speed_dayhr_past2wk = complete_time_perveh(self.past2wk_sensor_dayhr).\
        sort_values(['regno_clean','datetime_eat'])
       
        self.past2wk_totext_vio_perday = complete_time_perveh(self.past2wk_totext_vio_perday, 'date', 'D').\
        sort_values(['regno_clean','date'])
    
            
    
        # calculate total and extreme violations per 100 km per day
        self.sd_totext_vio_past2wk = self.past2wk_sensor_day
      
        # remove the entries where the daily distance is NA
        self.sd_totext_vio_past2wk = self.sd_totext_vio_past2wk[~(
            self.sd_totext_vio_past2wk.distance_hourly_sum_km.isnull())]
                                         
    
        #self.totext_vio_per100km_past2wk = self.sd_totext_vio_past2wk.\
        #    groupby(['regno_clean','route','date']).apply(lambda x: pd.Series({
        #        'totvio_per_100km': (x['N_valueg_above0_3'].sum()/x['distance_hourly_sum_km'].sum())*100,
        #        'extvio_per_100km': (x['N_valueg_above0_4'].sum()/x['distance_hourly_sum_km'].sum())*100
        #        })).reset_index()
    
        #self.totext_vio_per100km_past2wk = complete_time_perveh(self.totext_vio_per100km_past2wk, 
        #timevar = 'date', timefreq = 'D').sort_values(['regno_clean','date'])
    
    
        ## remove the entries where the distance is below 10
        self.sd_totext_vio_past2wk_thre =  self.sd_totext_vio_past2wk[
                                 self.sd_totext_vio_past2wk.distance_hourly_sum_km > 10]
        
        self.totext_vio_per100km_past2wk_thre = self.sd_totext_vio_past2wk_thre.\
        groupby(['regno_clean','route','date']).apply(lambda x: pd.Series({
            'totvio_per_100km': (x['N_valueg_above0_3'].sum()/x['distance_hourly_sum_km'].sum())*100,
            'extvio_per_100km': (x['N_valueg_above0_4'].sum()/x['distance_hourly_sum_km'].sum())*100
        })).reset_index()
        
        self.totext_vio_per100km_past2wk_thre = complete_time_perveh(self.totext_vio_per100km_past2wk_thre, 
                                        timevar = 'date', timefreq = 'D').sort_values(['regno_clean','date'])
                            
        
        ### summary table for past two weeks
        def first_last_date(df, data_name):
            ## get the first date each matatu is online
            first_online_date = df.sort_values(["regno_clean","date"]).groupby('regno_clean').\
            head(1)[["regno_clean","date"]].rename(columns = {'date': data_name +'1st_oldate'})
            ## get the last date each matatu is online
            last_online_date = df.sort_values(["regno_clean","date"]).groupby('regno_clean').\
            tail(1)[["regno_clean","date"]].rename(columns = {'date': data_name + 'last_oldate'})
            online_date = first_online_date.merge(last_online_date)
            
            return online_date
        
        # get the first and the last date each matatu is online in eco data
        self.online_date_eco = first_last_date(self.sensor_day, 'eco')
        
        # get the first and the last date each matatu is online in sensor data
        self.online_date_sensor = first_last_date(self.sensor_day[~(self.sensor_day['speed_min'].isna())], 'sns')
        
          
        viocnt = self.past2wk_sensor_day.groupby(['regno_clean']).apply(lambda x: pd.Series({
            'total_vio': x['N_valueg_above0_3'].sum(), 'ext_vio': x['N_valueg_above0_4'].sum(),
            'acc_totcnt': x['N_valueg_above0_3_acceleration'].sum(), 'acc_extcnt':x['N_valueg_above0_4_acceleration'].sum(),
            'turn_totcnt': x['N_valueg_above0_3_turn'].sum(), 'turn_extcnt':x['N_valueg_above0_4_turn'].sum(),
            'brake_totcnt': x['N_valueg_above0_3_brake'].sum(), 'brake_extcnt':x['N_valueg_above0_4_brake'].sum()
            })).reset_index()
        
        
        self.summary_table1 = viocnt.merge(self.online_date_sensor).merge(self.online_date_eco)
        
        sensor_1stlastday_past2wk = self.past2wk_sensor_day[~(self.past2wk_sensor_day['speed_min'].isna())]
        
        self.past2wk_online_date_eco =  first_last_date(self.past2wk_sensor_day, 'eco')
        self.past2wk_online_date_eco['eco_ndays'] = (self.past2wk_online_date_eco['ecolast_oldate'] - self.past2wk_online_date_eco['eco1st_oldate']).\
        dt.days + 1
        
        self.past2wk_online_date_sensor = first_last_date(sensor_1stlastday_past2wk, 'sns')
        self.past2wk_online_date_sensor['sns_ndays'] = (self.past2wk_online_date_sensor['snslast_oldate'] - \
        self.past2wk_online_date_sensor['sns1st_oldate']).dt.days + 1
        
        self.summary_table_final = self.summary_table1.merge(self.past2wk_online_date_eco[['regno_clean','eco_ndays']]).\
            merge(self.past2wk_online_date_sensor[['regno_clean','sns_ndays']])
    
    
    # -------------------------------------------------------------------------
    # all data figures   
    def hist_totvio_perwk(self, export = True):
            fig = px.bar(self.totext_vio_perwk, x="week", y = "total_vio" ,color = "week", 
                   facet_col="regno_clean",facet_col_wrap = 4,
                   labels = {"total_vio":"Totvio"},
                   facet_row_spacing=0.015, # default is 0.07 when facet_col_wrap is used
                   facet_col_spacing=0.04,# default is 0.03
                   height=2800, width=1100)
        
                   
            fig.update_layout(title_text="Number of total violation per week for all vehicles")
            
            if export:
                file_name = self.outputs_path + '/' + 'hist_totvio_wk_alldata.html'
                print('Saving: ' + file_name)
                plotly.offline.plot(fig, filename = file_name, auto_open=False)
                
                div = plotly.offline.plot(fig, filename = file_name, auto_open=False,
                                    include_plotlyjs=False, output_type='div')
                
                return div
                
    
    # -------------------------------------------------------------------------
    # past two weeks figures
    def hist_viotype_past2wk(self, export = True):
        melt_df = pd.melt(self.past2wk_viotype_cnt,id_vars=['regno_clean'],
                          value_vars =['brake','turn','acceleration'])
        
        fig = px.bar(melt_df, x="variable", y = "value" ,color = "variable", 
                   facet_col="regno_clean",facet_col_wrap = 4,
                   labels = {"variable":"Harsh event", "value":"cnt"},
                   title = "Past two weeks: Number of violation types for all vehicles",
                   facet_row_spacing=0.015,facet_col_spacing=0.04,height=2000, width=1100)
        
        if export:
            file_name = self.outputs_path + "/hist_viotypecnt_past2wk.html"
            print('Saving: ' + file_name)
            plotly.offline.plot(fig, filename = file_name, auto_open=False)

            div = plotly.offline.plot(fig, filename = file_name, auto_open=False,
                                    include_plotlyjs=False, output_type='div')
                
            return div
    
    
    def line_totalext_vio_past2wk(self,export = True):
        melt_df = pd.melt(self.past2wk_totext_vio_perday, id_vars=['regno_clean', 'date'],
                          value_vars =['total_vio','ext_vio'])
        
        fig = px.line(melt_df, x='date', y='value', color = 'variable',
                    facet_col ='regno_clean', facet_col_wrap = 4, 
                    facet_row_spacing=0.015, 
                    facet_col_spacing=0.04, 
                    height=2000, width=1200,
                    labels = {'value':'cnt'},
                    title= "Past two weeks: Number of violations for all vehicles")
        fig.update_yaxes(rangemode="tozero")
        #fig.update_yaxes(matches=None)
            
        
        if export:
            file_name = self.outputs_path + '/' + 'total_extvio_past2wk.html'
            print('Saving: ' + file_name)
            plotly.offline.plot(fig, filename = file_name, auto_open=False)

            div = plotly.offline.plot(fig, filename = file_name, auto_open=False,
                                    include_plotlyjs=False, output_type='div')
                
            return div
    
    
    def line_totalext_vio_per100km_past2wk(self, export = True):
        melt_df = pd.melt(self.totext_vio_per100km_past2wk,
                        id_vars = ['regno_clean','date'], 
                        value_vars = ['totvio_per_100km','extvio_per_100km'])
        
        fig = px.line(melt_df, x='date', y='value', color = 'variable',
                    facet_col ='regno_clean', facet_col_wrap = 4, 
                    facet_row_spacing=0.015, 
                    facet_col_spacing=0.04, 
                    height=2300, width=1200,
                    labels = {'value':'cnt'},
                    title= "Past two weeks: Number of violations per 100km for all vehicles")
        fig.update_yaxes(rangemode="tozero")
            #fig.update_yaxes(matches=None)
            
        
        if export:
            file_name = self.outputs_path + '/' + 'total_extvio_per_100km_past2wk2.html'
            print('Saving: ' + file_name)
            plotly.offline.plot(fig, filename = file_name, auto_open=False)
            
            div = plotly.offline.plot(fig, filename = file_name, auto_open=False,
                                    include_plotlyjs=False, output_type='div')
        
            
            
            return div
    
    
    def line_totalext_vio_per100km_past2wk_thre(self, export = True):
        melt_df = pd.melt(self.totext_vio_per100km_past2wk_thre,
                        id_vars = ['regno_clean','date'], 
                        value_vars = ['totvio_per_100km','extvio_per_100km'])
        
        fig = px.line(melt_df, x='date', y='value', color = 'variable',
                    facet_col ='regno_clean', facet_col_wrap = 4, 
                    facet_row_spacing=0.015, 
                    facet_col_spacing=0.04, 
                    height=2300, width=1200,
                    labels = {'value':'cnt'},
                    title= "Past two weeks: Number of violations per 100km for all vehicles (without outliers in distance)")
        fig.update_yaxes(rangemode="tozero")
            #fig.update_yaxes(matches=None)
            
        
        if export:
            file_name = self.outputs_path + '/' + 'total_extvio_per_100km_past2wk2_without_outlier.html'
            print('Saving: ' + file_name)
            plotly.offline.plot(fig, filename = file_name, auto_open=False)
            
            div = plotly.offline.plot(fig, filename = file_name, auto_open=False,
                                    include_plotlyjs=False, output_type='div')
        
            
            
            return div
                
    
    
    def line_speed85_dayhr_route_past2wk(self, export = True):
        uniq_route = self.speed_dayhr_past2wk.route.unique()
        div_list = []
        for r in uniq_route:
            sp_df = self.speed_dayhr_past2wk[self.speed_dayhr_past2wk.route == r]
            fig = px.line(sp_df, x='datetime_eat', y = "speed_p85",
                   facet_col="regno_clean", facet_col_wrap = 4, 
                   labels = {'datetime_eat': 'day and hour', 'speed_p85':' 85th speed'},
                   title =  r + ": 85th percentile of speed each day each hour",
                   facet_row_spacing=0.04, facet_col_spacing=0.04, height=1000, width=1100
                  )

            fig.update_traces(connectgaps=False)
            
            file_name = self.outputs_path + '/line_85thspeed_hr_past2wk_' + r +'.html'
            if export:
                print('Saving: ' + file_name)
                plotly.offline.plot(fig, filename = file_name, auto_open=False)
                div = plotly.offline.plot(fig, filename = file_name, auto_open=False,
                                    include_plotlyjs=False, output_type='div')
                
                div_list.append(div)
                
        return div_list
    
    
    def summary_table(self, export = True):
        fig = go.Figure(data=[go.Table(
        header=dict(values=list(self.summary_table_final.columns),
                fill_color='paleturquoise',
                align='left'),
        cells=dict(values=self.summary_table_final.transpose().values.tolist(),
               fill_color='lavender',
               align='left'))
        ])

        fig.update_layout(title_text="Past two weeks:\nSummary of the data")
        fig.update_layout(width=1700, height=1000)

        
        if export:
            file_name = self.outputs_path + '/' + 'report_viotype_spd_dt_past2wk.html'
            print('Saving: ' + file_name)
            plotly.offline.plot(fig, filename = file_name, auto_open=False)
            div = plotly.offline.plot(fig, filename = file_name, auto_open=True,
                                    include_plotlyjs=False, output_type='div')
            return div
    
    
    # -------------------------------------------------------------------------
    # create a dashboard and export as a html file. The date in the title and filename need to 
    # change everytime
    def return_all_figure_div(self):
        div1 = self.hist_totvio_perwk()
        div2 = self.hist_viotype_past2wk()
        div3 = self.line_totalext_vio_past2wk()
        #div4 = self.line_totalext_vio_per100km_past2wk()
        div7 = self.line_totalext_vio_per100km_past2wk_thre()
        div5 = self.line_speed85_dayhr_route_past2wk()
        div6 = self.summary_table()
        
        
        html_string = '''
        <html>
            <head>
            <script src="https://cdn.plot.ly/plotly-latest.min.js"></script>
            </head>
            <body>
              <h1>Weekly Report: 6/08/2022 - 6/21/2022</h1>
              
              ''' + div1 + '''
              ''' + div2 + '''
              ''' + div3 + '''
        
              ''' + div7 + '''
              ''' + div5[0] + '''
              ''' + div5[1] + '''
              ''' + div5[2] + '''
              ''' + div5[3] + '''
              ''' + div5[4] + '''
              ''' + div5[5] + '''
              ''' + div6 + '''
          
              
              
      
            </body>
        </html>'''

        with open(self.outputs_path + "/" + "weekly_report_20220608-20220621.html", 'w') as f:
            f.write(html_string)

In [8]:
report = weekly_report(folder_path,outputs_path,2022,6,21)

Loading data...
Aggregating data...


In [9]:
report.return_all_figure_div()

Saving: C:/Users/wb575963/OneDrive - WBG/Tasks/Kenya/Weekly_report_20220608-20220621/hist_totvio_wk_alldata.html
Saving: C:/Users/wb575963/OneDrive - WBG/Tasks/Kenya/Weekly_report_20220608-20220621/hist_viotypecnt_past2wk.html
Saving: C:/Users/wb575963/OneDrive - WBG/Tasks/Kenya/Weekly_report_20220608-20220621/total_extvio_past2wk.html
Saving: C:/Users/wb575963/OneDrive - WBG/Tasks/Kenya/Weekly_report_20220608-20220621/total_extvio_per_100km_past2wk2_without_outlier.html
Saving: C:/Users/wb575963/OneDrive - WBG/Tasks/Kenya/Weekly_report_20220608-20220621/line_85thspeed_hr_past2wk_Missing.html
Saving: C:/Users/wb575963/OneDrive - WBG/Tasks/Kenya/Weekly_report_20220608-20220621/line_85thspeed_hr_past2wk_Nairobi to Eldoret.html
Saving: C:/Users/wb575963/OneDrive - WBG/Tasks/Kenya/Weekly_report_20220608-20220621/line_85thspeed_hr_past2wk_Nairobi to Meru.html
Saving: C:/Users/wb575963/OneDrive - WBG/Tasks/Kenya/Weekly_report_20220608-20220621/line_85thspeed_hr_past2wk_Nairobi to Nakuru.html